# **STAT 301 Project Final Report**


### <font color=red> Online News Popularity
#### -  Bokai Lai, Sophia Oh, Can Okten, Rita Zhuang

# Introduction

In recent times, digital platforms have been primarily used as a source of news. According to Pew Research Center, 86% of U.S. adults often or sometimes access digital platforms for news, and among digital platforms, news websites and apps are most preferred. An existing scientific publication has used shareworthiness, a characteristic of online content to spread quickly, considering geographic proximity, cultural distance, and positivity/negativity of content to predict shares (Trilling et al., 2017). However, our group chose the “Online News Popularity” dataset from UCI Machine Learning Repository（https://archive.ics.uci.edu/dataset/332/online+news+popularity), which includes features about articles published by the global news website Mashable. There are many factors influencing how the public receives and shares content, and our goal is to build an effective model to predict the number of shares an article gets based on its characteristics.


Articles on Mashable can be directly shared on Facebook, Twitter, and Flipboard.  	The dataset contains articles that Mashable published between 2013 and 2015. There are 39,797 observations and 61 variables. To answer a prediction question, we will analyze shares as the response variable. Shares are an integer between 0 and infinite, representing the overall popularity and reach of an article.

# Methods and Results

## Exploratory Data Analysis (EDA)

In [1]:
install.packages("Hmisc")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
install.packages("psych")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [ ]:
library(tidyverse)
library(repr)
library(readxl)
library(infer)
library(cowplot)
library(GGally)
library(broom)
library(dplyr)
library(car)
library(tidymodels)
library(glmnet)
library(leaps)
library(faraway)
library(mltools)
library(caret)
library(forcats)
library(gridExtra)
library(patchwork)
library(scales)
library(utils)
library(httr)
library(Hmisc)
library(psych)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp


Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2

Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some


── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ dials        1

In [ ]:
ONP_raw <- read.csv('https://raw.githubusercontent.com/jasonllai/Stat-301-project-t36/main/OnlineNewsPopularity.csv', header = TRUE, sep = ",")
head(ONP_raw)

*Table 2.1 First 6 rows of raw data*

In [ ]:
# Select variables of interest
selected_data <- ONP_raw %>%
  select(3,4,8,10,12,28,57,45,46,47,48,51,54,58,61)

head(selected_data)

*Table 2.2 First 6 rows of data interested*

| Column | Variable Name | Type | Meaning |
|--------|-----------------------|-----|----|
| 1      | `n_tokens_title`        | type numeric `dbl`| Number of words in the title |
| 2      | `n_tokens_content`      | type numeric `dbl`| Number of words in the content |
| 3      | `num_hrefs`             | type numeric `dbl`| Number of links |
| 4      | `num_imgs`              | type numeric `dbl`| Number of images |
| 5     | `average_token_length`  | type numeric `dbl`| Average length of the words in the content |
| 6     | `kw_avg_avg`            | type numeric `dbl`| Average number of shares (popularity) of the average keyword included in the article (keywords are similar to hashtags) |
| 7     | `global_subjectivity`   | type numeric `dbl`| Text subjectivity |
| 8     | `global_sentiment_polarity` | type numeric `dbl`| Text sentiment polarity |
| 9     | `global_rate_positive_words` | type numeric `dbl`| Rate of positive words in the content |
| 10     | `global_rate_negative_words` | type numeric `dbl`| Rate of negative words in the content |
| 11     | `avg_positive_polarity` | type numeric `dbl`| Average polarity of positive words |
| 12     | `avg_negative_polarity` | type numeric `dbl`| Average polarity of negative words |
| 13     | `title_subjectivity`    | type numeric `dbl`| Title subjectivity |
| 14     | `title_sentiment_polarity` | type numeric `dbl` | Title sentiment polarity |
| 15     | `shares`                | type numeric `int`| Number of shares on Facebook, Twitter, and Flipboard [RESPONSE VARIABLE] |


### Research question:
How can we build an effective model to predict the number of shares an article gets based on the following variables: number of words in the title (`n_tokens_title`), number of images (`num_imgs`), number of words in content (`n_tokens_content`), number of links (`num_hrefs`), average length of words in the content (`average_token_length`), title subjectivity (`title_subjectivity`), global subjectivity (`global_subjectivity`), global sentiment polarity (`global_sentiment_polarity`), rate of positive words (`rate_positive_words`), rate of negative words (`rate_negative_words`), negative polarity (`avg_negative_polarity`), positive polarity (`avg_positive_polarity`), title sentiment polarity (`title_sentiment_polarity`), and average keywords (`kw_avg_avg`)?

### Cleaning and Wrangling

#### Integer Values

Most of our variables are integer values (counts) without decimals (such as number of images, words, links, etc.). We will check which variables need transformation using `sapply()`.

In [ ]:
# Assuming df is your data frame
result <- sapply(selected_data, function(x) all(x == floor(x)))
                 
# Convert integer columns
for (col in names(selected_data)) {
    if (result[col]) {
        selected_data[[col]] <- as.integer(selected_data[[col]])
    }
}              
str(selected_data)

In [ ]:
# Readable column names
news_data <- selected_data %>%
    rename(n_words_title = n_tokens_title,
           n_words_content = n_tokens_content,
           n_links = num_hrefs,
           n_images = num_imgs,
           avg_word_length = average_token_length,
           avg_keyword_popularity = kw_avg_avg,
           subjectivity_title = title_subjectivity,
           subjectivity_text = global_subjectivity,
           sentiment_polarity_text = global_sentiment_polarity,
           sentiment_polarity_title = title_sentiment_polarity,
           rate_positive_words_text = global_rate_positive_words,
           rate_negative_words_text = global_rate_negative_words,
           avg_positive_polarity = avg_positive_polarity,
           avg_negative_polarity = avg_negative_polarity
          )
str(news_data)

### Explanatory Visualization

#### Basic Summary Statistics

We will start by exploring the surface level characteristics of the data such as the variable types, number of rows, columns and check if the dataset contains any missing (NA) values.

We will then use the `describe()` function to explore the basic summary statistics including measures of central tendency and measures of dispersion. This will give us a good grasp on where the center of the variables are located and how spread out the values are within those variables.

In [ ]:
dims <- dim(news_data)
rows <- dims[1]
columns <- dims[2]

cat("\nDimensions and missing values:\n")
# Dimensions and NA check
if (sum(is.na(news_data)) == 0){
    print(paste0("The dataset consists of ", as.character(rows), " observations and ",  as.character(columns), " variables (", 
                 as.character(columns-1), " + 1 response variable). There are no missing values in the dataset."))
}

In [ ]:
cat("\nBasic summary statistics table - Measures of Central Tendency and Measures of Dispersion:\n")
description <- round(psych::describe(news_data), 2)

description$variance <- description$sd^2

round(description, 2)

*Table 2.3 Measures of Central Tendency and Measures of Dispersion*

#### Observations:
In _**Poisson regression**_ it's important that the mean equals the variance for the count data. The variance for `shares` is significantly greater than the mean. This is called _overdispersion_ and it needs to be investigated further. This can be due to the excess of zeros in these data or the nature of count (`int`) data.  

Since some of the count variables can be true 0's, we will only focus on the `n_words_content` and `avg_word_length` variables. This is because we expect the articles we'll be using in the analysis to have at least some word content in them. We will check the articles with the `url` column in the original dataset. If they are geniune data we will keep the 0's, if not, we will remove them.

In [ ]:
# Subset data of articles with suspicious word counts and their links
no_word_articles <- ONP_raw %>% filter(n_tokens_content == 0 | average_token_length == 0) %>% select(url, shares)
head(no_word_articles, 3)

*Table 2.4 Subset data of articles with suspicious word counts and their links*

After analyzing some of the articles, we found that they do have words in them (words not equal to 0) and thus, their average word lengths shouldn't be equal to 0. These are erroneous entries, and wil be removed from the data. 

In [ ]:
# Data entry errors removed
news_data <- news_data %>% filter(n_words_content != 0, avg_word_length != 0)

### Correlation Matrix

We will continue the EDA with a correlation matrix. This will help us have a better idea of both the correlations between variables and with the response variable `shares`. 

**NOTE:** Any correlation below 0.5 will be assigned 'NA' to highlight the notable correlations only.

In [ ]:
# Create correlation matrix
cor_matrix <- cor(news_data[,-1])
threshold <- 0.5

cor_matrix[abs(cor_matrix) < threshold] <- NA
round(cor_matrix, 2)

*Table 2.5 correlation matrix*

#### Variance Inflation Factor (VIF) Analysis

Since Poisson Regression doesn't account for multicollinearity, we will do a VIF analysis to detect if any correlations are problematic.

In [ ]:
lm_vif_model <- lm(shares ~., data = news_data)

vif <- vif(lm_vif_model)

print(round(vif, 3))

We observe that only the `sentiment_polarity_text` can be considered problematic, with a **VIF score of  6.508**. We will drop this variable before continuing the analysis. 

In [ ]:
news_data <- news_data %>% select(-sentiment_polarity_text )

In [ ]:
lm_vif_model2 <- lm(shares ~., data = news_data)

vif2 <- vif(lm_vif_model2)

print(round(vif2, 3))

### Visualizations

Next up, we will use `ggplot2::ggpairs()` function to visualize these relationships using correlations and scatterplots for continuous variables. 

Due to the heavy skew observed in the summary statistics (**_skew = 33.96_**) and the extremely large range of the `shares` varaible (**_range = 843299_**) given the relatively small mean of this variable (**_mean = 3395.38_**), we will first filter the number of shares to less than 20,000. This will help viusalize the response variable more clearly. 

Trimmed dataset where the `shares` variable is trimmed to remove outliers will be assigned to a new variable called `news_data_trim`.

We will take a sample of 250 using `sample_n()` for computational purposes and we will use `set.seed()` function for reproducibility, before visualizing with `ggpairs()`.

In [ ]:
set.seed(123)
# Trim response variable
news_data_trim <- news_data %>% filter(shares < 20000)

# Sample for correlations
sample_250 <- news_data_trim %>% sample_n(250)

In [ ]:
ggpairs(sample_250[,c(1:5, 14)], progress = FALSE,
        lower = list(continuous = wrap("points", alpha = 0.2)))

*Figure 2.1 First subset of correlation matrix*

In [ ]:
ggpairs(sample_250[,c(6:14)], progress = FALSE,
       lower = list(continuous = wrap("points", alpha = 0.1)))

*Figure 2.2 Second subset of correlation matrix*

### Histogram Plots

We now want to have a look at the distribution of the response variable `shares`. We use geom_histogram to plot the distribution.

In [ ]:
# Sample for correlations
sample_1000 <- news_data_trim %>% sample_n(1000)

options(repr.plot.width = 8, repr.plot.height = 4)
shares_dist <- sample_1000 %>%
    ggplot(aes(x = shares)) +
    geom_histogram(bins = 50, color = "white") +
    xlim(0,20000) +
    labs(x = "Number of shares", y = "Count") +
    ggtitle("Distribution of Number of shares") +
    theme(text = element_text(size = 10)) +
    theme(plot.title = element_text(hjust = 0.45))
shares_dist

*Figure 2.3 Distribution of Number of shares*

This graph shows a heavily right skewed distribution of our interested response variable (`shares`). 

We noticed that there are 2 rows removed from this graph, this is because these data points are lied on the very right of the hostogram and their counts are very low. Since the grpah is to visualize the distribution of number of shares, we can temporarily ignore them here.

## Model Selection and Prediction

In this project, we want to compare two models: `Ridge Regression` and `Poisson Regression`. We will perform cross-validation to compare two models.

### Splitting Training Data and Testing Data

In [ ]:
news_data <- news_data %>% relocate(shares)

# Split data into training and test sets
set.seed(123)

training_ONP  = news_data %>%
  sample_frac(0.6)

testing_ONP = news_data %>%
  setdiff(training_ONP)

head(training_ONP,6)
head(testing_ONP,6)

*Table 2.6 First six rows for training set and testing set*

### Building Ridge Regression Model

To train our Ridge Regression model, we will use `glmnet` package, which requires a matrix with input variables and a vector of responses. Thus, we use as.matrix function to prepare the model matrix for glmnet. We create 4 matrices, which are the training x-matrix `ONP_X_train` and training y-matrix `ONP_Y_train`; the testing x-matrix `ONP_X_test` and testing y-matrix `ONP_Y_test`.

In [ ]:
# Build matrix and vector required by `glmnet`

ONP_X_train <- model.matrix(object = shares ~ .,
  data = training_ONP)[, -1]

ONP_Y_train <- training_ONP[, "shares"]

ONP_X_test <- model.matrix(object = shares ~ .,
  data = testing_ONP)[, -1]

ONP_Y_test <- testing_ONP[, "shares"]

With the training data `ONP_X_train` and `ONP_Y_train`, we will use `cv.glmnet()` function to find an "optimal" value of $\hat{\lambda}$ value for our Ridge model. In `cv.glmnet()` function, we set `alpha = 0` as we want to get a Ridge model. We want to see the results and find the value $\hat{\lambda}_{min}$.

In [ ]:
ONP_cv_lambda_ridge <- cv.glmnet(
  x = ONP_X_train, y = ONP_Y_train,
  alpha = 0,
  lambda = exp(seq(-5, 10, 0.1))
)

Now, we will examine the result of the cross-validation `ONP_cv_lambda_ridge`

In [ ]:
ONP_cv_lambda_ridge

We can see that $\hat{\lambda}_{\text{1SE}}$ here is very large, which slightly implies that Ridge Regression might not be very suitable for our data. However, we will use $\hat{\lambda}_{min}$ to build our Ridge Model and name it `ONP_cv_lambda_ridge`.

In [ ]:
ONP_cv_lambda_ridge <- glmnet(
  x = ONP_X_train, y = ONP_Y_train,
  alpha = 0,
  lambda = ONP_cv_lambda_ridge$lambda.min
)

### Building Poisson Regression Model

The reason we build Poisson Regression Model is because a Poisson random variable takes discrete non-negative integer values that count something in a given timeframe. Our dataset has number of shares as response variable which is also a discrete non-negative variable counting the number of shares in a period of 2 years. This characteristic of our response variable meets all the traits of a Poisson random variable.

In order to fit a Poisson regression model, we can use the function `glm()` and its argument `family = poisson`, which obtains the estimates $\hat{\beta}_0, \hat{\beta}_1, \dots \hat{\beta}_{p}$. The estimates are obtained through maximum likelihood where we assume a Poisson joint probability mass function of the $n$ responses $Y_i$.

Here we use `shares` as response variable and all other variables as explanatory variables to build a Poisson regression model, name it `ONP_Poisson_model`. 

In [ ]:
ONP_Poisson_model <- glm(
  formula = shares ~.,
  data = training_ONP,
  family = poisson
)
summary(ONP_Poisson_model)

### Model Comparison

To select the best model for the dataset to predict number of shares, we use `caret` package with `glmnet` to perform a 10-fold CV for comparing models. Below is the specific code:

In [ ]:
# Step 1: Prepare data
predictors <- subset(training_ONP, select = -c(shares)) # Features
target <- training_ONP$shares # Target variable

# Step 2: Create the models (Ridge regression and Poisson regression)
# Ridge regression using glmnet package
model_ridge <- train(predictors, target, method = "glmnet", 
                     trControl = trainControl(method = "cv", number = 10),
                     tuneGrid = expand.grid(alpha = 0, lambda = seq(-5, 10, by = 0.1)), 
                     family = "gaussian")

# Poisson regression using glm package
model_poisson <- train(predictors, target, method = "glm", 
                       trControl = trainControl(method = "cv", number = 10),
                       tuneLength = 5,
                       family = poisson())

# Step 3: Perform cross-validation to compare models
compareModels <- resamples(list(Ridge = model_ridge, Poisson = model_poisson))

# Summarize and compare the performance of models
summary(compareModels)

From the summary above, $MAE$, $RMSE$, and $R^2$ values are calculated for both models. We will focus on analyzing the mean for each metrics for both models. We can see that the $MAE$ for Ridge Regression Model is slightly lower than($\approx$ 1) $MAE$ of Poisson Regression Model. The $RMSE$ for Ridge Regression Model is a lot higher than ($\approx$ 3687) $RMSE$ of Poisson Regression Model. The $R^2$ for Ridge Regression Model is slightly higher than($\approx$ 0.002) $R^2$ of Poisson Regression Model. Since the $MAE$ and $R^2$ metric values for both models are about the same whereas the $RMSE$ of Poisson Regression Model is much less than that of Ridge Regression Model, **we can conclude that Poisson Regression Model is better for us to use to predict number of shares in a period of two years.**

### Prediction

We use `predict()` function and the Poisson regression model `ONP_Poisson_model` to obtain the out-of-sample predicted values of number of shares of an article in a period of two years using testing set `testing_ONP`. We store the predicted values in a variable called `test_pred_poisson`. We will look at the first few predicted value using `head()` function.

In [ ]:
test_pred_poisson <- predict(ONP_Poisson_model, newdata = testing_ONP,
                             type = "response")
head(test_pred_poisson)

Use the function `rmse()` to compute the  RMSE$_{test}$ using the predicted values stored in `test_pred_poisson`.

In [ ]:
ONP_R_MSE_models <- tibble(
  Model = "Poisson Regression",
  R_MSE = rmse(
    preds = test_pred_poisson,
    actuals = testing_ONP$shares
  )
)
ONP_R_MSE_models

*Table 2.7 RMSE of predicted values by Poisson Regression*

The RMSE value above indicates that the average difference between the number of shares predicted by Poisson model and the actual number of shares is approximately 12582. 

# Discussion

### Summary

In our analysis, we explored Ridge Regression and Poisson Regression. Ridge Regression uses an L2-norm to measure the size of the coefficients, which is the sum of the squares of each coefficient. It is appropriate because our dataset has many explanatory variables to consider, but none of them are strongly correlated with the response variable. Poisson Regression is also an appropriate model to use because our response variable is a count. This method takes discrete non-negative integer values, and the Poisson distribution has a mean equal to its variance.

We first checked the **Linearity assumption** by plotting the residuals and observing randomness. Next, we ensured that all errors were independent to fulfill the **Independence assumption**. After that, we checked the **Constant Variance assumption**, which assumes that the errors have equal variance. Randomness in the residuals-fitted value plot indicates that heteroskedasticity is not present. One major issue we expected to face was *multicollinearity* because there are many explanatory variables, some of which could be related. We used a correlation matrix and VIF to quantify multicollinearity, and dropped the variable sentiment_polarity_text to continue with our analysis. Furthermore, there are no missing values in the dataset. The distribution of shares had a heavy right-skew, so we concluded that articles with more than 20,000 shares were outliers and safely ignored them.

Although Ridge can be used to address multicollinearity problems, one limitation is that it does not select variables since the estimated coefficients won't be shrunk to zero. Another limitation is that the coefficients may be biased. However, this is not a major concern because having biased coefficients does not affect our predictions. Our lambda.min value of 1097 gives the value with the minimum mean cross-validation error. Our lambda.1se value of 22026 gives the value such that the cross-validation error is within one standard error of the minimum, which is about 10 times larger than the lambda.min value. This corresponds with a higher level of penalization, and may imply that Ridge Regression might not be very suitable for our data.

For Poisson Regression, any factor that affects the mean will also affect the variance, which could be a potential drawback for using this model. Residual deviance measures how much probabilities estimated from our model differ from the observed proportions of successes. In our case, the value is 135116829, which suggests a very large difference. This means the Poisson method may not produce the best model either.

### Key Findings

After performing a 10-fold cross validation to compare Ridge and Poisson Regression Models, we concluded that the **Poisson Regression Model is better to predict article shares.** However, the Poisson Regression’s RMSE value based on the test dataset produced a value of 12582, which is relatively large for the difference between the number of shares. This means that the **Poisson Regression model may not be the best overall model to analyze this dataset.**


### Improvements and Future Research

Although it is beyond the scope of STAT 301, a **Negative Binomial Regression model** may produce a better model for our dataset because it is more sensitive to heavy skew in the data. Also, we could have used other proportions (0.5, 0.7, etc.) to split the data and see which proportion gives us the best model. A future question we have is whether we can generalize this prediction model. For instance, can this model be applied to articles from other news sites, such as the New York Times or Washington Post? Can it be generalized to other digital platforms, including Instagram and Reddit? Other sites and platforms may affect how articles are shared, so more research and analysis are needed to determine external validity.

# References

Trilling, D., Tolochko, P., & Burscher, B. (2017). From Newsworthiness to Shareworthiness: 
How to Predict News Sharing Based on Article Characteristics. Journalism & Mass 
Communication Quarterly, 94(1), 38-60. https://doi.org/10.1177/1077699016654682

Fernandes, K., Vinagre, P., Cortez, P., & Sernadela, P. (2015, May 30). Online News Popularity. UCI Machine Learning Repository. https://archive.ics.uci.edu/dataset/332/online+news+popularity

Liedke, J., & Wang, L. (2023, November 15). News Platform Fact Sheet. Pew Research Center’s Journalism Project. https://www.pewresearch.org/journalism/fact-sheet/news-platform-fact-sheet/#:~:text=The%20transition%20of%20the%20news,they%20are%20getting%20their%20news. 